# 環境設定

In [ ]:
# ! pip install google-cloud-bigquery

In [8]:
import os
from google.cloud import bigquery

# SET GCP 金鑰
os.environ['GOOGLE_APPLICATION_CREDENTIALS']=r"/content/tibame-gad253-14-bigquery-c5993cbf5beb.json"

# 初始化BigQuery客戶端
client = bigquery.Client()
print("Client creating using default project: {}".format(client.project))

Client creating using default project: tibame-gad253-14-bigquery


In [9]:
# 設定資料集
dataset_id = 'tibame_gad253_14_dataset_python'
us_dataset_id = 'tibame_gad253_14_dataset_us'
eu_dataset_id = 'tibame_gad253_14_dataset_eu'

# 資料庫操作

## 刪除資料表

In [3]:
table_id = 'age_table'
table_ref = client.dataset(dataset_id).table(table_id)
table = client.get_table(table_ref)
client.delete_table(table)
print("Deleted table '{}'.".format(table_ref.path))

Deleted table '/projects/tibame-gad253-14-bigquery/datasets/tibame_gad253_14_dataset_python/tables/age_table'.


## 刪除資料集

In [4]:
dataset_ref = client.dataset(dataset_id)
client.delete_dataset(dataset_ref, delete_contents=True, not_found_ok=True)
print("Deleted dataset '{}'.".format(dataset_id))

Deleted dataset 'tibame_gad253_14_dataset_python'.


# 邏輯回歸模型

## 建立分割資料檢視表

In [7]:
inputdata = 'input_data'
# 定義SQL查詢建立檢視表
query = f"""
CREATE OR REPLACE VIEW`{us_dataset_id}.{inputdata}`AS
SELECT
  age,
  workclass,
  marital_status,
  education_num,
  occupation,
  hours_per_week,
  income_bracket,
  CASE
    WHEN MOD(functional_weight, 10) < 8 THEN 'training'
    WHEN MOD(functional_weight, 10) = 8 THEN 'evaluation'
    WHEN MOD(functional_weight, 10) = 9 THEN 'prediction'
  END AS dataframe
FROM
  `bigquery-public-data.ml_datasets.census_adult_income`
"""
# 執行查詢
query_job = client.query(query)
query_job.result()

## 建立模型
https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-create#create_model_syntax

In [22]:
rog_model='census_model'
# 定義SQL查詢建立模型
query=f"""
CREATE OR REPLACE MODEL `{us_dataset_id}.{rog_model}`
OPTIONS
  (model_type='LOGISTIC_REG',
    input_label_cols=['income_bracket'],
    data_split_method='NO_SPLIT',
    max_iterations=15) AS
SELECT
  * EXCEPT(dataframe)
FROM
  `{us_dataset_id}.{inputdata}`
WHERE
  dataframe = 'training'
"""
# 印出查詢
print(query)

# 執行SQL查詢
query_job=client.query(query)
query_job.result()


CREATE OR REPLACE MODEL `tibame_gad253_14_dataset_us.census_model`
OPTIONS
  (model_type='LOGISTIC_REG',
    input_label_cols=['income_bracket'],
    data_split_method='NO_SPLIT',
    max_iterations=15) AS
SELECT
  * EXCEPT(dataframe)
FROM
  `tibame_gad253_14_dataset_us.input_data`
WHERE
  dataframe = 'training'



## 評估分類模型

In [27]:
# 定義SQL查詢進行模型評估
query = f"""
SELECT
*
FROM
ML.EVALUATE(MODEL `{us_dataset_id}.{rog_model}`,
(
    SELECT * EXCEPT(dataframe)
    FROM `{us_dataset_id}.{inputdata}`
    WHERE dataframe = 'evaluation'
))
"""
# 印出Query
print(query)
# 執行查詢
query_job = client.query(query)
results = query_job.result()


SELECT
*
FROM
ML.EVALUATE(MODEL `tibame_gad253_14_dataset_us.census_model`,
(
    SELECT * EXCEPT(dataframe)
    FROM `tibame_gad253_14_dataset_us.input_data`
    WHERE dataframe = 'evaluation'
))



In [28]:
# 將results的row轉成tuple
rows_data = [tuple(row.values()) for row in results]
# 抓出results的欄位名稱
colums = [field.name for field in results.schema]

import pandas as pd
df = pd.DataFrame(rows_data, columns=colums)
df.head()

,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,0.687609,0.535326,0.8385,0.601986,0.350941,0.882538


## 測試模型

In [32]:
# 定義SQL測試邏輯回歸模型
query = f"""
SELECT
income_bracket,	predicted_income_bracket,predicted_income_bracket_probs
FROM
ML.PREDICT(MODEL `{us_dataset_id}.{rog_model}`,
(
    SELECT * EXCEPT(dataframe)
    FROM `{us_dataset_id}.{inputdata}`
    WHERE dataframe = 'evaluation'
))
"""
# 印出Query
print(query)
# 執行查詢
query_job = client.query(query)
results = query_job.result()


SELECT 
income_bracket,	predicted_income_bracket,predicted_income_bracket_probs	
FROM
ML.PREDICT(MODEL `tibame_gad253_14_dataset_us.census_model`,
(
    SELECT * EXCEPT(dataframe)
    FROM `tibame_gad253_14_dataset_us.input_data`
    WHERE dataframe = 'evaluation'
))



In [33]:
# 將results的row轉成tuple
rows_data = [tuple(row.values()) for row in results]
# 抓出results的欄位名稱
colums = [field.name for field in results.schema]

import pandas as pd
df = pd.DataFrame(rows_data, columns=colums)
df.head()

,income_bracket,predicted_income_bracket,predicted_income_bracket_probs
0,<=50K,<=50K,"[{'label': ' >50K', 'prob': 0.206070047304485}, {'label': ' <=50K', 'prob': 0.793929952695515}]"
1,<=50K,<=50K,"[{'label': ' >50K', 'prob': 0.10104946095684697}, {'label': ' <=50K', 'prob': 0.898950539043153}]"
2,<=50K,<=50K,"[{'label': ' >50K', 'prob': 0.09351356752496746}, {'label': ' <=50K', 'prob': 0.9064864324750326}]"
3,>50K,<=50K,"[{'label': ' >50K', 'prob': 0.11541413828687455}, {'label': ' <=50K', 'prob': 0.8845858617131255}]"
4,<=50K,<=50K,"[{'label': ' >50K', 'prob': 0.08552628451719632}, {'label': ' <=50K', 'prob': 0.9144737154828037}]"


In [ ]:
# 取消自動截斷
pd.set_option('display.max_colwidth', None)
df.head(1)# 顯示第一筆資料

## 解釋測試結果

In [ ]:
#定義SQL查詢解釋模型測試
query=f"""
SELECT
*
FROM
ML.EXPLAIN_PREDICT(MODEL `{us_dataset_id}.{rog_model}`,
  (
    SELECT * EXCEPT(dataframe)
    FROM `{us_dataset_id}.{inputdata}`
    WHERE dataframe = 'prediction'
  ),STRUCT(6 AS top_k_features)
)
"""
#印出查詢
print(query)

#執行查詢
query_job=client.query(query)
results=query_job.result()

#將results的row解成tuple
rows_data= [tuple(row.values()) for row in results]
#抓出results欄位名稱
columns=[field.name for field in results.schema]
#import panda
import pandas as pd
#顯示資料
df=pd.DataFrame(rows_data,columns=columns)
#取消自動截斷
pd.set_option('display.max_colwidth', None)

In [38]:
df.head(1)

,predicted_income_bracket,probability,top_feature_attributions,baseline_prediction_value,prediction_value,approximation_error,age,workclass,marital_status,education_num,occupation,hours_per_week,income_bracket
0,<=50K,0.914641,"[{'feature': 'education_num', 'attribution': -1.2248251714178617}, {'feature': 'occupation', 'attribution': -1.2001341890977935}, {'feature': 'marital_status', 'attribution': 0.6247716603288829}, {'feature': 'workclass', 'attribution': -0.49893779358740464}, {'feature': 'age', 'attribution': 0.38374421016600013}, {'feature': 'hours_per_week', 'attribution': -0.013021959314922322}]",-0.443256,-2.371659,0.0,52,Local-gov,Married-civ-spouse,5,Other-service,40,<=50K


## 全局模型解釋

In [ ]:
# 定義SQL查詢建立全局解釋模型
query=f"""
CREATE OR REPLACE MODEL `{us_dataset_id}.{rog_model}`
OPTIONS
  (model_type='LOGISTIC_REG',
    input_label_cols=['income_bracket'],
    data_split_method='NO_SPLIT',
    max_iterations=15,
    auto_class_weights=TRUE,
    enable_global_explain=TRUE
    ) AS
SELECT
  * EXCEPT(dataframe)
FROM
  `{us_dataset_id}.{inputdata}`
WHERE
  dataframe = 'training'
"""
# 印出查詢
print(query)

# 執行查詢
query_job=client.query(query)
query_job.result()

## 全局特徵影響解釋

In [ ]:
# 定義SQL查詢全局特徵解釋
query=f"""
SELECT
*
FROM
ML.GLOBAL_EXPLAIN(MODEL `{us_dataset_id}.{rog_model}`)
"""
# 印出查詢
print(query)

# 執行查詢
query_job=client.query(query)
results=query_job.result()

# 將results的row解成tuple
rows_data= [tuple(row.values()) for row in results]
# 抓出results欄位名稱
columns=[field.name for field in results.schema]
# import panda
import pandas as pd
# 顯示資料
df=pd.DataFrame(rows_data,columns=columns)
# 取消自動截斷
pd.set_option('display.max_colwidth', None)

In [42]:
df

,feature,attribution
0,marital_status,1.178824
1,education_num,0.569518
2,occupation,0.520015
3,age,0.381932
4,workclass,0.310469
5,hours_per_week,0.264324
